In [2]:
import retrieval_search
import json, os
from openai import OpenAI

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'subject': '3ng-tech'},
        boost_dict=boost,
        num_results=2
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
        Eres un asistente conversacional. Responde la PREGUNTA basado en el CONTEXTO de la base de datos.

        PREGUNTA: {question}

        CONTEXTO: 
        {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"seccion: {doc['section']}\npregunta: {doc['question']}\nrespuesta: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [5]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

In [8]:
for subject_docs in docs_raw:
    for doc in subject_docs['documents']:
        doc['subject'] = subject_docs['subject']
        documents.append(doc)

In [9]:
index = retrieval_search.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["subject"]
)

index.fit(documents)

In [11]:
client = OpenAI()

In [13]:
rag("Como es fisicamente el CEO de 3ng?")

'Basado en el contexto proporcionado, físicamente el CEO de 3ng podría no ser considerado atractivo, ya que se menciona que es "el más feo de 3ng".'